# Statistics and figures

## How many tasks per number of keywords

In [1]:
import pandas as pd

In [2]:
file_keywords = "../data/neural_network_y.csv"
df = pd.read_csv(file_keywords)

# count number of keywords used per task, by adding an extra column
numeric_cols = df.select_dtypes(include="number")
df["row_sum"] = numeric_cols.sum(axis=1)
df = df.sort_values(by="row_sum", ascending=True)

for i in range(0, 11) :
    df_s = df[df["row_sum"] == i]
    task_names = df_s["task_id"].values
    print("For %d keywords, found a total of %d tasks: %s" % (i, df_s.shape[0], str(task_names)))

For 0 keywords, found a total of 0 tasks: []
For 1 keywords, found a total of 0 tasks: []
For 2 keywords, found a total of 7 tasks: ['67a3c6ac' 'a416b8f3' '68b16354' 'ed36ccf7' '6150a2bd' '74dd1130'
 '3c9b0459']
For 3 keywords, found a total of 8 tasks: ['8be77c9e' 'c59eb873' 'c9e6f938' '6fa7a44f' '6d0aefbc' '4c4377d9'
 '9172f3a0' '7468f01a']
For 4 keywords, found a total of 8 tasks: ['b1948b0a' 'b91ae062' '5582e5ca' 'f25fbde4' 'd511f180' '2dee498d'
 'd10ecb37' 'c8f0f002']
For 5 keywords, found a total of 6 tasks: ['62c24649' '8d5021e8' '6f8cd79b' 'c1d99e64' '3af2c5a8' '67e8384a']
For 6 keywords, found a total of 6 tasks: ['25ff71a9' '5bd6f4ac' '28bf18c6' '46442a0e' '46f33fce' '7fe24cdd']
For 7 keywords, found a total of 5 tasks: ['dc1df850' '60b61512' 'a740d043' '7b7f7511' '6d75e8bb']
For 8 keywords, found a total of 12 tasks: ['42a50994' '3aa6fb7a' '9565186b' '88a62173' '4258a5f9' '0b148d64'
 'd631b094' 'ac0a08a4' 'f76d97a5' 'a79310a0' 'e9614598' '56ff96f3']
For 9 keywords, found a t